In [9]:
import os
import utils as ut
import torch
import numpy as np
from torch import nn
from torchvision.models.alexnet import alexnet


In [10]:
class AlexNetConv5(nn.Module):
    def __init__(self):
        super().__init__()

        base = alexnet(pretrained=True)
        self.conv = base.features
        self.conv1 = base.features[:2]
        self.avgpool = base.avgpool
        
        self.eval()

    def forward(self, stimuli):
        x = self.conv(stimuli) #shape: [stim, 256, 7, 7] / mean test set r = 0.7297
#         x = self.conv1(x) #shape: [stim, 256, 6, 6]
#         x = self.avgpool(x) #shape: [stim, 256, 6, 6]
        x = x.view(x.size(0), -1) #shape: [stim, 12544] / mean test set r =  0.7218
        #x = self.fc_6(x) #shape: [stim, 4096] / mean test set r = 0.7142
        return x

In [20]:
img = ut.image_to_tensor('../../../aardvark_01b.jpg',resolution = 227)
img = img.unsqueeze(0) 
model = AlexNetConv5()

In [21]:
with torch.no_grad():
    feats = model(img).cpu().numpy()

In [22]:
feats.shape

(1, 9216)

In [23]:
AlexNetConv5()

AlexNetConv5(
  (conv): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))


In [24]:
class AlexNet_cm(nn.Module):
    def __init__(self, feature_name, pretrained_val = True):
        super().__init__()
        self.feature_name = feature_name
        base = alexnet(pretrained=pretrained_val)
        self.conv_1 = base.features[:3]
        self.conv_2 = base.features[3:6]
        self.conv_3 = base.features[6:8]
        self.conv_4 = base.features[8:10]
        self.conv_5 = base.features[10:]
        self.avgpool = base.avgpool
        self.fc_1 = base.classifier[:3]
        self.fc_2 = base.classifier[3:6]
        self.fc_3 = base.classifier[6:]
        self.eval()
    def forward(self, stimuli):
        x = self.conv_1(stimuli)
        if 'conv_1' == self.feature_name: return x.view(x.shape[0], -1)
        x = self.conv_2(x)
        if 'conv_2' == self.feature_name: return x.view(x.shape[0], -1)
        x = self.conv_3(x)
        if 'conv_3' == self.feature_name: return x.view(x.shape[0], -1)
        x = self.conv_4(x)
        if 'conv_4' == self.feature_name: return x.view(x.shape[0], -1)
        x = self.conv_5(x)
        if 'conv_5' == self.feature_name: return x.view(x.shape[0], -1)
        x = self.avgpool(x)
        x = x.view(x.shape[0], -1)
        if 'pool' == self.feature_name: return x
        x = self.fc_1(x)
        if 'fc_1' == self.feature_name: return x
        x = self.fc_2(x)
        if 'fc_2' == self.feature_name: return x
        x = self.fc_3(x)
        if 'fc_3' == self.feature_name: return x
        return None 

In [25]:
img = ut.image_to_tensor('../../../aardvark_01b.jpg',resolution = 227)
img = img.unsqueeze(0) 

In [27]:
model_cm = AlexNet_cm('conv_5')

In [28]:
with torch.no_grad():
    feats_cm = model_cm(img).cpu().numpy()

In [45]:
np.corrcoef(feats, feats_cm)

array([[1., 1.],
       [1., 1.]])